In [2]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator,CrossValidatorModel
from pyspark.ml.recommendation import ALS, ALSModel

In [3]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [3]:
sameModel = ALSModel.load("C:/Users/uttas/Desktop/Resume/HBCALSModel2")

In [4]:
sameModel = ALSModel.load("C:/Users/uttas/Desktop/Resume/HBCALSModel4")

In [5]:
type(sameModel)

pyspark.ml.recommendation.ALSModel

In [6]:
file_path = 'C:/Users/uttas/Desktop/Resume/HBC/Customer_Tran_Sample.csv'

In [7]:
sdf = spark.read.csv(file_path, header=True, inferSchema=True)

In [8]:
sdf = sdf.withColumnRenamed('CUSTOMER_ID', 'customer_id')
sdf = sdf.withColumnRenamed('UPC_ID', 'upc_id')
sdf = sdf.withColumnRenamed('ITEM_QTY', 'quantity')
sdf = sdf.withColumnRenamed('DOLLAR_AMT', 'dollar_amt')
sdf = sdf.withColumnRenamed('DIV_CD', 'div_cd')
sdf = sdf.withColumnRenamed('TRAN_DT', 'date')
sdf = sdf.withColumnRenamed('BRAND_ID', 'brand_id')

In [9]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
sdf2 = sdf.withColumn("unique_customer_id", F.dense_rank().over(Window.orderBy(sdf.customer_id)))

In [10]:
sdf3 = sdf2.withColumn("unique_product_id", F.dense_rank().over(Window.orderBy(sdf.upc_id)))

In [11]:
sdf4 = sdf3.withColumn("brand_id", F.dense_rank().over(Window.orderBy(sdf.brand_id)))

In [12]:
cols = ['unique_customer_id','unique_product_id','quantity','brand_id']
sdf5 = sdf4.select(*cols)

In [13]:
(train, test) = sdf5.randomSplit([0.7, 0.3])

In [38]:
brand_1 = sdf5.filter(sdf5['unique_product_id']==22214).select(['brand_id'])

In [39]:
brand_1.show()

+--------+
|brand_id|
+--------+
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
+--------+
only showing top 20 rows



In [40]:
brand_2 = sdf5.filter(test['unique_product_id']==16353).select(['brand_id'])

In [41]:
brand_2.show()

+--------+
|brand_id|
+--------+
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
|     107|
+--------+
only showing top 20 rows



In [14]:
test_predictions = sameModel.transform(test)

In [15]:
test_predictions.show()

+------------------+-----------------+--------+--------+------------+
|unique_customer_id|unique_product_id|quantity|brand_id|  prediction|
+------------------+-----------------+--------+--------+------------+
|               148|            16987|       1|      75|5.4509757E-5|
|               463|            31044|       1|     118|3.6671823E-5|
|               471|              933|       1|     132|  0.03735495|
|               471|             4893|       1|     129|2.0741501E-4|
|               471|             6108|       1|     129|1.7203266E-6|
|               471|             6120|       1|     129|5.0555172E-6|
|               471|            19890|       1|     142|1.2736322E-4|
|              1088|              215|       2|     123|  0.14126737|
|              1238|             5715|       1|     147|3.2938927E-4|
|              1580|             7005|       1|      94| 3.685258E-6|
|              1580|             7005|       1|      94| 3.685258E-6|
|              1591|

In [16]:
nrecommendations = sameModel.recommendForAllUsers(5)

In [17]:
from pyspark.sql.functions import col, explode
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('unique_customer_id', col("rec_exp.unique_product_id"))
    
nrecommendations.limit(10).show()

+------------------+-----------------+
|unique_customer_id|unique_product_id|
+------------------+-----------------+
|               148|            13035|
|               148|            18436|
|               148|            16353|
|               148|            16893|
|               148|            21152|
|               463|            24226|
|               463|            11138|
|               463|            24749|
|               463|            24295|
|               463|            11069|
+------------------+-----------------+



In [18]:
nrecommendations_1591 = nrecommendations.filter(nrecommendations['unique_customer_id']==1591).select(['unique_product_id'])

In [19]:
nrecommendations_1591.show()

+-----------------+
|unique_product_id|
+-----------------+
|            13035|
|            18436|
|            16353|
|            21152|
|            16893|
+-----------------+



In [20]:
test_1591 = test.filter(test['unique_customer_id']==1591).select(['unique_product_id','quantity'])

In [21]:
test_1591.show()

+-----------------+--------+
|unique_product_id|quantity|
+-----------------+--------+
|              895|       1|
|            18434|       1|
|            21227|       1|
|            21577|       1|
|            22213|       1|
|            22214|       1|
+-----------------+--------+

